In [7]:
import findspark
import pyspark
import random
from pyspark.sql.functions import max, min
from pyspark.mllib.recommendation import ALS


In [10]:
findspark.init('')
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
user_artist = sc.textFile("user_artist_data_small.txt").map(lambda line : line.split(" ")).map(lambda x : (int(x[0]),int(x[1]),int(x[2])))
user_artist.take(5)
#df = data.map(line => (user,artist,) = line.split(' ')).toDF("User","Artist")
#user_artist_df = user_artist.toDF(["User","Artist","Count"])
#user_artist_df.agg(min("User")).show()

artist = sc.textFile("artist_data_small.txt").map(lambda line : line.split("\t")).map(lambda x : (int(x[0]),x[1]))
artist.take(5)
#artist_df = artist.toDF(["ArtistID","Name"])
#artist_df.show()

artist_alias = sc.textFile("artist_alias_small.txt").map(lambda line : line.split("\t")).map(lambda x : (int(x[0]),int(x[1])))
artist_alias_map = artist_alias.collectAsMap()
data_correct_artist = user_artist.map(lambda x: (x[0], artist_alias_map.get(x[1], x[1]), x[2]))

In [ ]:
trainData, validationData, testData = data_correct_artist.randomSplit([0.4,0.4,0.2], seed=100)
trainData.cache()
validationData.cache()
testData.cache()

In [ ]:
unique_artist = data_correct_artist.map(lambda x : x[1]).distinct().collect()
uniq_art = sc.broadcast(unique_artist)

In [ ]:


def model_eval(model, data):
    all_artist = uniq_art.value
    unique_users = validationData.map(lambda x: x[0]).distinct().collect()
    train_User_Artist = dict(trainData.map(lambda x:(x[0],x[1])).groupByKey().map(lambda x: (x[0],list(x[1]))).collect())
    test_User_Artist =  dict(validationData.map(lambda x:(x[0],x[1])).groupByKey().map(lambda x: (x[0],list(x[1]))).collect()) 
    total = 0.0
    for user in unique_users:
        true_artist = set(test_User_Artist[user])
        non_train_artist = set(all_artist) - set(train_User_Artist[user]) 
        test_data= []
        for artist in non_train_artist:
            test_data.append((user,artist))

        test_data_rdd = sc.parallelize(test_data)
        predictions = best_model.predictAll(test_data_rdd)
        X = len(true_artist)
        topX = sorted(predictions.collect(), key = lambda x : x[2], reverse = True)[:X]
        topX_RDD = sc.parallelize(topX)
        topArtist = set(topX_RDD.map(lambda x:x[1]).collect())
        total += float(len((true_artist) & set(topArtist)))/float(len((true_artist)))
    return total/len(unique_users)

In [ ]:
rank = [2,7,20]
for val in rank:
    best_model = ALS.trainImplicit(trainData,rank = val, seed = 345)
    score = model_eval(best_model,validationData)
    print(score)


In [ ]:

model_eval(best_model,testData)

best_model.recommendProducts(1059334,5)